In [1]:
import cv2
import mediapipe as mp
import math
import serial
import time
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)  # Allow 2 hands
mp_draw = mp.solutions.drawing_utils

# Initialize serial connection to ESP32
ser = serial.Serial('/dev/ttyUSB0', 115200)  # Replace with your ESP32 port
time.sleep(2)  # Wait for connection

# Finger landmark indices (MediaPipe)
INDEX_FINGER_TIP = 8
INDEX_FINGER_PIP = 6  # Proximal Interphalangeal Joint
INDEX_FINGER_MCP = 5  # Metacarpophalangeal Joint
WRIST = 0

def calculate_angle(a, b, c):
    """Calculate angle between three points (in degrees)"""
    # Convert points to tuples if they're landmark objects
    a = (a.x, a.y) if hasattr(a, 'x') else a
    b = (b.x, b.y) if hasattr(b, 'x') else b
    c = (c.x, c.y) if hasattr(c, 'x') else c
    
    # Calculate vectors
    ba = (a[0] - b[0], a[1] - b[1])
    bc = (c[0] - b[0], c[1] - b[1])
    
    # Calculate dot product and magnitudes
    dot_product = ba[0]*bc[0] + ba[1]*bc[1]
    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    mag_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    
    # Avoid division by zero
    if mag_ba * mag_bc < 0.001:
        return 0.0
    
    # Calculate angle in radians, then convert to degrees
    angle_rad = math.acos(max(-1.0, min(1.0, dot_product / (mag_ba * mag_bc))))
    angle_deg = math.degrees(angle_rad)
    
    return angle_deg

cap = cv2.VideoCapture(0)
prev_left_angle = 90
prev_right_angle = 90

while cap.isOpened():
    success, img = cap.read()
    if not success:
        continue
        
    img = cv2.flip(img, 1)  # Mirror image for more intuitive control
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    
    # Initialize servo angles to neutral position
    left_servo_angle = 90
    right_servo_angle = 90
    
    if results.multi_hand_landmarks:
        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            # Get landmarks
            lm = hand_landmarks.landmark
            
            # Draw hand landmarks
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Determine handedness (left or right hand)
            handedness = results.multi_handedness[hand_idx].classification[0].label
            
            # Calculate index finger angle
            finger_angle = calculate_angle(
                lm[INDEX_FINGER_MCP], 
                lm[INDEX_FINGER_PIP],
                lm[INDEX_FINGER_TIP]
            )
            
            # Map angle to servo range (0°-180°)
            servo_angle = int(np.interp(finger_angle, [0, 90], [0, 180]))
            
            # Simple smoothing to reduce jitter
            if handedness == "Left":
                left_servo_angle = int(0.7 * servo_angle + 0.3 * prev_left_angle)
                left_servo_angle = max(0, min(180, left_servo_angle))
                prev_left_angle = left_servo_angle
                
                # Display left hand info
                cv2.putText(img, f"Left: {int(finger_angle)}deg -> Servo: {left_servo_angle}", 
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                
            elif handedness == "Right":
                # Reverse mapping for right hand if needed
                right_servo_angle = int(0.7 * servo_angle + 0.3 * prev_right_angle)
                right_servo_angle = max(0, min(180, right_servo_angle))
                prev_right_angle = right_servo_angle
                
                # Display right hand info
                cv2.putText(img, f"Right: {int(finger_angle)}deg -> Servo: {right_servo_angle}", 
                            (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    # Send angles to ESP32
    ser.write(f"{left_servo_angle},{right_servo_angle}\n".encode())
    
    # Display instructions
    cv2.putText(img, "Left hand controls left servo", (10, img.shape[0] - 80), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(img, "Right hand controls right servo", (10, img.shape[0] - 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(img, "Press 'q' to quit", (10, img.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    cv2.imshow("Dual-Hand Servo Control", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
ser.write("90,90\n".encode())  # Reset servos to neutral
time.sleep(0.5)
ser.close()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1755352746.993486   16553 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755352747.004208   16558 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/rustom/anaconda3/envs/aienv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
